In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score, log_loss, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import shap
from tqdm import tqdm
import optuna
import lightgbm as lgb
import pickle
from itertools import combinations
from textwrap import wrap
optuna.logging.set_verbosity(optuna.logging.WARNING)

# データ読み込み
normal_data = pd.read_csv('normal_features_few_light.csv')
abnormal_data = pd.read_csv('anormal_features_few5_light.csv')

normal_data['label'] = 0
abnormal_data['label'] = 1
data = pd.concat([normal_data, abnormal_data], ignore_index=True)

# Exclude list
Exclude_list = ["hashvin", "total_ig_cycle"] # "hashvin", "total_ig_cycle"

# 特徴量とターゲット,特徴名リスト
X = data.drop(['label'] + Exclude_list, axis=1)
y = data['label']
feature_names = X.columns.tolist()

# 基本情報
num_features = X.shape[1]
num_normal = normal_data.shape[0]
num_abnormal = abnormal_data.shape[0]
total_samples = num_normal + num_abnormal
target_num_feature = int(total_samples * 0.1)
ffr = target_num_feature / num_features
ffr = 1 #ffr if ffr < 1 else 1.0
print(f"特徴量数:{num_features}, 目標:{target_num_feature}, 正常:{num_normal}, 異常:{num_abnormal}, 特徴量/全データ:{num_features / total_samples * 100:.2f}%, ffr:{ffr}\n")

# 故障index
abnormal_indices = y[y == 1].index

# 設定値
run_optimization = False
n_trials = 200
estop = 20
threshold = 0.6 # shap閾値

default_params = {
'num_leaves': 100,
'min_data_in_leaf': 25,
'learning_rate': 0.05,
'reg_alpha': 7,
}

optimize_params = ['num_leaves', 'min_data_in_leaf', 'learning_rate', 'reg_alpha'] #'reg_alpha'

if total_samples < 5000:
    num_leaves_range = (20, 200)
    mdil = (20, 30)
elif total_samples < 50000:
    num_leaves_range = (31, 350)
    mdil = (20, 500)
else:
    num_leaves_range = (50, 500)
    mdil = (20, 1000)

logloss_per_trial = []
f1_scores_all_trials = []

def objective(trial):
    params = {}

    for param_name in optimize_params:
        if param_name == 'num_leaves':
            params[param_name] = trial.suggest_int(param_name, num_leaves_range[0], num_leaves_range[1])
        elif param_name == 'min_data_in_leaf':
            params[param_name] = trial.suggest_int(param_name, mdil[0], mdil[1])
        elif param_name == 'learning_rate':
            params[param_name] = trial.suggest_float(param_name, 0.01, 0.5, log=True)
        elif param_name == 'reg_alpha':
            params[param_name] = trial.suggest_float(param_name, 3, 10)

    params.update({
        'objective': 'binary',
        'metric': 'binary_logloss',
        'feature_fraction': ffr,
        'num_boost_round': 2000,
        'scale_pos_weight': num_normal / num_abnormal,
        'verbose': -1,
        **default_params
    })

    kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    logloss_scores = []
    f1_scores = []
    for train_index, test_index in kf.split(X, y):

        X_train = X.iloc[train_index]
        X_test = X.iloc[test_index]

        y_train = y.iloc[train_index]
        y_test = y.iloc[test_index]

        lgb_train = lgb.Dataset(X_train, y_train)
        lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)

        evals_result = {}
        model = lgb.train(
            params,
            lgb_train,
            valid_sets=[lgb_train, lgb_eval],
            valid_names=['train', 'valid'],
            callbacks=[
                lgb.early_stopping(stopping_rounds=estop, verbose=False, first_metric_only=True),
                lgb.record_evaluation(evals_result)
            ]
        )

        preds = model.predict(X_test)
        logloss = log_loss(y_test, preds)
        logloss_scores.append(logloss)

        preds_binary = (preds > 0.5).astype(int)
        f1 = f1_score(y_test, preds_binary)
        f1_scores.append(f1)

    mean_logloss = np.mean(logloss_scores)
    mean_f1 = np.mean(f1_scores)

    logloss_per_trial.append(mean_logloss)
    f1_scores_all_trials.append(mean_f1)

    return mean_f1

# 最適化
if run_optimization:
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=n_trials)

    best_params = study.best_params
    best_f1_score = study.best_value
    min_logloss = min(logloss_per_trial)
    average_f1_score = np.mean(f1_scores_all_trials)

    print("\n最小logloss:", min_logloss)
    print("最大F1:", best_f1_score)
    print("平均F1:", average_f1_score)

if run_optimization:
    best_params = study.best_params
else:
    best_params = {param_name: default_params[param_name] for param_name in default_params}

print(f"best_params:{best_params}")

opt_params = {
'objective': 'binary',
'metric': ['binary_logloss', 'f1'],
'feature_fraction': ffr,
'num_boost_round': 2000,
'scale_pos_weight': num_normal / num_abnormal,
'verbose': -1,
**default_params
}

# 最適paraがあれば置き換え
if run_optimization:
    opt_params.update(best_params)

print(f"opt_params:{opt_params}\n")

n_splits = 5
n_repeats = 10
kf = StratifiedKFold(n_splits=n_splits, shuffle=True,) #random_state=42

fold_results = []
train_loss_curves = []
valid_loss_curves = []
confusion_matrices = []

for repeat_index in range(n_repeats):
    for fold, (train_index, test_index) in enumerate(kf.split(X, y), start=1):

        X_train = X.iloc[train_index]
        X_test = X.iloc[test_index]

        y_train = y.iloc[train_index]
        y_test = y.iloc[test_index]

        lgb_train = lgb.Dataset(X_train, y_train)
        lgb_eval = lgb.Dataset(X_test, y_test, reference=lgb_train)
        evals_result = {}
        model = lgb.train(
            opt_params,
            lgb_train,
            valid_sets=[lgb_train, lgb_eval],
            valid_names=['train', 'valid'],
            callbacks=[
                lgb.early_stopping(stopping_rounds=estop, verbose=False, first_metric_only=True),
                lgb.record_evaluation(evals_result)
            ]
        )

        preds = model.predict(X_test)
        preds_binary = (preds > 0.5).astype(int)
        cm = confusion_matrix(y_test, preds_binary)
        confusion_matrices.append(cm) 
        precision = precision_score(y_test, preds_binary)
        recall = recall_score(y_test, preds_binary)
        f1 = f1_score(y_test, preds_binary)
        importances = model.feature_importance(importance_type='gain')
        importance_df = pd.DataFrame({'feature': feature_names, 'importance': importances})
        importance_df = importance_df.sort_values('importance', ascending=False)

        fold_results.append({
            'repeat': repeat_index + 1,
            'fold': fold,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'importance_df': importance_df,
            'train_loss_curve': evals_result['train']['binary_logloss'],
            'valid_loss_curve': evals_result['valid']['binary_logloss'],
        })

# 学習曲線
metrics = ['Binary Log Loss', ]
fig, axes = plt.subplots(1, 5, figsize=(20, 4))
top_5_folds = sorted(fold_results, key=lambda x: x['f1'], reverse=True)[:5]

for idx, fold_result in enumerate(top_5_folds):
    train_data = fold_result['train_loss_curve']
    valid_data = fold_result['valid_loss_curve']

    iterations = range(1, len(train_data) + 1)
    axes[idx].plot(iterations, train_data, label='Train')
    axes[idx].plot(iterations, valid_data, label='Validation')
    axes[idx].set_ylabel('Binary Log Loss')
    axes[idx].set_xlabel('Iteration')
    axes[idx].set_title(f"Repeat {fold_result['repeat']} Fold {fold_result['fold']} - Binary Log Loss")
    axes[idx].legend()

plt.tight_layout()
plt.show()

# Confusion Matrix
average_cm = np.mean(confusion_matrices, axis=0)
plt.figure(figsize=(5, 4))
sns.heatmap(average_cm, annot=True, fmt=".2f", cmap="Blues") 
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Ave Confusion Matrix")
plt.show()

# 評価指標,重要度
avg_precision = np.mean([result['precision'] for result in fold_results])
avg_recall = np.mean([result['recall'] for result in fold_results])
avg_f1 = np.mean([result['f1'] for result in fold_results])
print(f"Ave Precision: {avg_precision:.2f}")
print(f"Ave Recall(TPR): {avg_recall:.2f}")
print(f"Ave F1 Score: {avg_f1:.2f}")

avg_importance = pd.concat([result['importance_df'].set_index('feature') for result in fold_results], axis=1).mean(axis=1).reset_index()
avg_importance.columns = ['feature', 'importance']
avg_importance = avg_importance.sort_values('importance', ascending=False)

LGBM_top_features = avg_importance.head(10)
print("\nAverage Feature Imp(before_selection)")
for _, row in LGBM_top_features.iterrows():
    print(f"Feature: {row['feature']}, Importance: {row['importance']:.4f}")

# 重要度0の特徴削除
zero_importance_features = avg_importance[avg_importance['importance'] == 0]['feature'].tolist()
num_0imp = len(zero_importance_features)
X_dropped = X.drop(columns=zero_importance_features)
feature_names_dropped = X_dropped.columns.tolist()
print(f"\n削除特徴数: {num_0imp}")
print(f"残特徴数:{num_features-num_0imp}")
print(f"削減率：{num_0imp/num_features*100:.1f}%")

# All SHAP
X_train = X_dropped
Y_train = y
X_test = X_dropped
Y_test = y

lgb_train = lgb.Dataset(X_train, Y_train)
lgb_test = lgb.Dataset(X_test, Y_test, reference=lgb_train)

model = lgb.train(opt_params, lgb_train,
                valid_sets=[lgb_train, lgb_test],
                valid_names=['train', 'test'],
                callbacks=[lgb.early_stopping(stopping_rounds=estop, verbose=False, first_metric_only=True)], )

explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)

positive_shap_values = np.maximum(shap_values[1], 0)
mean_shap_all = np.mean(positive_shap_values, axis=0)
sorted_feature_indices_all = np.argsort(mean_shap_all)[::-1]

thresholded_positive_shap_values = np.where(shap_values[1] > threshold, shap_values[1], 0)
mean_shap_thresholded = np.mean(thresholded_positive_shap_values, axis=0)
sorted_feature_indices_thresholded = np.argsort(mean_shap_thresholded)[::-1]

new_order_indices = []
for idx in sorted_feature_indices_thresholded:
    if mean_shap_thresholded[idx] > 0: 
        new_order_indices.append(idx)

remaining_indices = [idx for idx in sorted_feature_indices_all if idx not in new_order_indices]
new_order_indices.extend(remaining_indices)

# top_10_features
top_10_features_idx = new_order_indices[:10]
top_10_features = [feature_names_dropped[i] for i in top_10_features_idx]

shap.initjs()
shap_exp = shap.Explanation(
    values=shap_values[1][:, new_order_indices],
    feature_names=[feature_names_dropped[i] for i in new_order_indices],
    data=X_test.iloc[:, new_order_indices],
)
shap.plots.beeswarm(shap_exp, max_display=11, order=np.arange(len(new_order_indices)))
plt.tight_layout()
plt.show()

# --- DP plot ---
num_cols = 5
num_rows = (len(top_10_features) + num_cols - 1) // num_cols
fig, axes = plt.subplots(num_rows, num_cols, figsize=(20, 4 * num_rows))

for i, feature_index in enumerate(top_10_features_idx):
    row = i // num_cols
    col = i % num_cols

    feature_shap_values = shap_values[1][:, feature_index]
    feature_values = X_test.iloc[:, feature_index]
    
    normal_indices = y[y == 0].index
    abnormal_indices = y[y == 1].index

    normal_shap_values = feature_shap_values[normal_indices]
    ratio = np.sum(normal_shap_values < 0) / np.sum(normal_shap_values >= 0)

    axes[row, col].axhline(y=0, color='black', linewidth=0.5)
    axes[row, col].axhline(y=threshold, color='gray', linestyle='--', linewidth=0.5, label='Threshold')

    axes[row, col].scatter(feature_values[normal_indices], 
                           feature_shap_values[normal_indices], 
                           alpha=0.5, c='blue', label='Normal')

    axes[row, col].scatter(feature_values[abnormal_indices], 
                           feature_shap_values[abnormal_indices], 
                           alpha=0.5, c='red', label='Anormal')
    
    mean_positive_shap = np.mean(feature_shap_values[feature_shap_values > threshold])
    axes[row, col].plot([], [], ' ', label=f'Mean: {mean_positive_shap:.2f}')
    axes[row, col].plot([], [], ' ', label=f'NRatio: {ratio:.1f}')

    axes[row, col].set_title(f'{feature_names_dropped[feature_index]}')
    axes[row, col].set_xlabel('Feature Value')
    axes[row, col].set_ylabel('Contribution')
    handles, labels = axes[row, col].get_legend_handles_labels()
    by_label = dict(zip(labels, handles))
    axes[row, col].legend(by_label.values(), by_label.keys())

for i in range(len(top_10_features), num_rows * num_cols):
    row = i // num_cols
    col = i % num_cols
    fig.delaxes(axes[row, col])

plt.tight_layout()
plt.show()

# --- フィルタリングと散布図のプロット ---
plt.figure(figsize=(20, 4))

filter_conditions = []  # フィルタ条件を格納するリスト (Plot 2 以降で累積的に使用)

for plot_index in range(1, 6):
    plt.subplot(1, 5, plot_index)

    # Plot 2 以降の場合、新しい特徴量の条件を追加
    if plot_index > 1:
        feature_rank = plot_index + 1  # 3位(index:2)の特徴から順番に処理
        feature_index = top_10_features_idx[feature_rank - 1]
        feature_name = feature_names_dropped[feature_index]

        # 異常クラスのshap値が閾値以上のデータを取得
        anormal_feature_values = X_dropped[y == 1][feature_name]
        anormal_shap_values = shap_values[1][y == 1, feature_index]

        valid_anormal_indices = np.where(anormal_shap_values > threshold)[0]  # 閾値を超えるインデックスを取得

        # 有効な異常データが存在する場合のみ範囲を決定
        if len(valid_anormal_indices) > 0:
            min_value = np.min(anormal_feature_values.iloc[valid_anormal_indices])
            max_value = np.max(anormal_feature_values.iloc[valid_anormal_indices])
            filter_conditions.append((feature_name, min_value, max_value))  # 条件リストに追加

    # タイトルにフィルタ条件を表示 (最大4行、左揃え)
    title = ""
    for i, condition in enumerate(filter_conditions):
        feature_name, min_value, max_value = condition
        title += f"{feature_name}: {min_value:.2f} < x < {max_value:.2f}"
        if i < len(filter_conditions) - 1:
            title += "\n"  # 改行を追加

    # タイトルを最大4行に wrap し、左揃えで表示
    title_lines = wrap(title, width=30)[:4]  # 30文字で改行
    plt.title("\n".join(title_lines), loc='left')  # 左揃えで表示

    # フィルタ条件を適用してデータを抽出
    filtered_X = X_dropped.copy()
    filtered_y = y.copy()
    for condition in filter_conditions:
        feature_name, min_value, max_value = condition
        filtered_X = filtered_X[(filtered_X[feature_name] > min_value) & (filtered_X[feature_name] < max_value)]
        filtered_y = filtered_y[filtered_X.index]  # インデックスを合わせてyもフィルタリング

    # データが存在する場合のみプロット
    if len(filtered_X) > 0:
        n_normal = len(filtered_X[filtered_y == 0])
        n_abnormal = len(filtered_X[filtered_y == 1])

        plt.scatter(filtered_X[filtered_y == 0][top_10_features[0]], filtered_X[filtered_y == 0][top_10_features[1]], c='blue', alpha=0.5, label=f'Normal ({n_normal})')
        plt.scatter(filtered_X[filtered_y == 1][top_10_features[0]], filtered_X[filtered_y == 1][top_10_features[1]], c='red', alpha=0.5, label=f'Anormal ({n_abnormal})')
        plt.xlabel(top_10_features[0])
        plt.ylabel(top_10_features[1])
        plt.legend()
    else:
        # データがない場合は空欄にする
        plt.plot([], [])  # 空のデータでプロット
        plt.xlabel(top_10_features[0])
        plt.ylabel(top_10_features[1])
        plt.title("No data matches\nthe filter conditions", loc='left')  # データがない場合のタイトル

plt.tight_layout()
plt.show()

# --shap.dependence_plot--
N = 6
top_N_features_idx = top_10_features_idx[:N]
comb_top_N = list(combinations(top_N_features_idx, 2))

num_cols = 5
num_rows = (len(comb_top_N) + num_cols - 1) // num_cols
fig, axes = plt.subplots(num_rows, num_cols, figsize=(20, 4 * num_rows))

for i, (feature1_idx, feature2_idx) in enumerate(comb_top_N):
    row = i // num_cols
    col = i % num_cols

    shap.dependence_plot(
        ind=feature_names_dropped[feature1_idx],
        shap_values=shap_values[1],
        features=X_test,
        feature_names=feature_names_dropped,
        interaction_index=feature_names_dropped[feature2_idx],
        ax=axes[row, col],
        show=False,)

    axes[row, col].axhline(y=0, color='black', linewidth=0.5)
    axes[row, col].axhline(y=threshold, color='gray', linestyle='--', linewidth=0.5)
    axes[row, col].set_xlabel(feature_names_dropped[feature1_idx])
    axes[row, col].set_ylabel('Contribution')

for i in range(len(comb_top_N), num_rows * num_cols):
    row = i // num_cols
    col = i % num_cols
    fig.delaxes(axes[row, col])

plt.tight_layout()
plt.show()

# --hist,scatter--
num_cols = 5
num_rows = (len(top_10_features) + num_cols - 1) // num_cols
fig, axes = plt.subplots(num_rows, num_cols, figsize=(20, 4 * num_rows))

for i, feature in enumerate(top_10_features):
    row = i // num_cols
    col = i % num_cols

    axes[row, col].hist(X_dropped[y == 0][feature], color='blue', alpha=0.5, label='Normal')
    axes[row, col].hist(X_dropped[y == 1][feature], color='red', alpha=0.5, label='Abnormal')
    axes[row, col].set_title(feature)

for i in range(len(top_10_features), num_rows * num_cols):
    row = i // num_cols
    col = i % num_cols
    fig.delaxes(axes[row, col])

plt.tight_layout()
plt.show()

num_cols = 5
num_rows = (len(list(combinations(top_10_features, 2))) + num_cols - 1) // num_cols
fig, axes = plt.subplots(num_rows, num_cols, figsize=(20, 4 * num_rows))

for i, feature_pair in enumerate(combinations(top_10_features, 2)):
    row = i // num_cols
    col = i % num_cols
    feature1, feature2 = feature_pair
    
    axes[row, col].scatter(X_dropped[y == 0][feature1], X_dropped[y == 0][feature2], c='blue', alpha=0.5, label='Normal')
    axes[row, col].scatter(X_dropped[y == 1][feature1], X_dropped[y == 1][feature2], c='red', alpha=0.5, label='Anormal') 
    axes[row, col].set_xlabel(feature1)
    axes[row, col].set_ylabel(feature2)
    axes[row, col].legend()

for i in range(len(list(combinations(top_10_features, 2))), num_rows * num_cols):
    row = i // num_cols
    col = i % num_cols
    fig.delaxes(axes[row, col])

plt.tight_layout()
plt.show()